In [1]:
import torch 
from torch import optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence

import matplotlib.pyplot as plt

from AutoEncoder.model import AutoModel
from AutoEncoder.utils import setData, trainSentence, collate_fn, predit, encode, decode

from dataset import VisDialDataset
from utils.token import Lang
from tqdm import tqdm

jsonFile = "/home/ball/dataset/mscoco/visdialog/visdial_1.0_train.json"
cocoDir = "/home/ball/dataset/mscoco/"
langFile = "dataset/lang.pkl"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

SyntaxError: invalid syntax (__init__.py, line 1)

In [2]:
lang = Lang.load(langFile)
dataset = VisDialDataset(dialFile = jsonFile,
                         cocoDir = cocoDir, 
                         sentTransform = torch.LongTensor,
#                          convertSentence = lang.sentenceToVector
                        )
# dataset = dataset.getAllSentences()

Load lang model: dataset/lang.pkl. Word size: 43974


Preparing image paths with image_ids: 133351it [00:00, 381220.58it/s]


In [4]:
import dataset

In [5]:
dataset.VisDialDataset

dataset.VisDialDataset.VisDialDataset

In [5]:
loader = torch.utils.data.DataLoader(dataset.getAllSentences(),
                                     batch_size=5, 
                                     shuffle=True, 
                                     num_workers=4,
                                     collate_fn=collate_fn)
it = iter(loader)

In [3]:
seqs = it.next()
en_seq, de_seq = setData(seqs, lang, DEVICE)

NameError: name 'it' is not defined

In [3]:
model = torch.load("AutoEncoder/models/trainset/Automodel.29.pth").to(DEVICE).eval()

In [28]:
score = []
sents = []
for i in tqdm(range(0000, 1000)):
    source = dataset.data["dialogs"][i]["caption"]
    decode = predit(model, DEVICE, lang, lang.sentenceToVector(source))[0]
    score.append(source == decode)
    sents.append([source, decode])

100%|██████████| 1000/1000 [00:18<00:00, 54.87it/s]


In [19]:
import numpy as np
print("Q")
np.array(score).mean()

Q


0.94199999999999995

In [21]:
import numpy as np
print("A")
np.array(score).mean()

A


0.79400000000000004

In [29]:
import numpy as np
print("cap")
np.array(score).mean()

cap


0.621

In [5]:
import h5py
f = h5py.File('visdial_train.h5', 'w')

In [6]:
caption_feature = []
for source in tqdm(dataset.data["dialogs"]):
    source = source["caption"]
    feature = encode(model, DEVICE, lang, lang.sentenceToVector(source))
    caption_feature.append(feature[0][0].detach().cpu().numpy())
f["caption"] = caption_feature

100%|██████████| 123287/123287 [02:30<00:00, 817.50it/s]


In [7]:
answers_feature = []
for source in tqdm(dataset.data["answers"]):
    feature = encode(model, DEVICE, lang, lang.sentenceToVector(source))
    answers_feature.append(feature[0][0].detach().cpu().numpy())
f["answers"] = answers_feature

100%|██████████| 337527/337527 [06:48<00:00, 827.04it/s]


In [8]:
questions_feature = []
for source in tqdm(dataset.data["questions"]):
    feature = encode(model, DEVICE, lang, lang.sentenceToVector(source))
    questions_feature.append(feature[0][0].detach().cpu().numpy())
f["questions"] = questions_feature

100%|██████████| 376082/376082 [07:41<00:00, 814.99it/s]


In [9]:
f.close()

In [13]:
fr = h5py.File('visdial_train.h5', 'r')

In [15]:
fr["questions"][0]

array([ -7.83710331e-02,   4.60426629e-01,  -2.75709391e-01,
         3.15831542e-01,   2.79123604e-01,   3.04227173e-01,
        -7.87969470e-01,  -1.10471603e-02,   1.95630774e-01,
         2.87877113e-01,  -1.33238673e-01,  -7.14917034e-02,
        -3.98376137e-01,  -1.68182269e-01,  -7.19684064e-01,
         1.46047354e-01,  -1.47766754e-01,   4.56336737e-01,
        -3.88716012e-01,  -1.61294162e-01,   1.21676373e+00,
        -4.58553284e-01,   8.64968002e-01,   3.92491311e-01,
         2.28496268e-01,   2.85082549e-01,   4.36866194e-01,
         1.99247241e-01,   2.14725077e-01,   8.70687962e-01,
        -5.93807042e-01,   4.15823996e-01,   4.14954722e-02,
        -6.93105757e-01,   1.65639855e-02,  -3.30827832e-02,
         4.12149847e-01,  -6.69730961e-01,  -2.22960502e-01,
         1.07980147e-01,  -4.94128883e-01,   5.16955018e-01,
        -1.52980179e-01,   7.86192179e-01,  -3.64582866e-01,
        -5.85264266e-02,  -2.47624964e-01,   3.84946354e-02,
         8.68316144e-02,